In [2]:
import sqlite3
import pandas as pd
import csv
import argparse

In [4]:
db = sqlite3.connect("Covid vaccines.db")
cur = db.cursor()


In [5]:
df = pd.read_csv("C:/Users/jenny/Documents/GitHub/python_programmering/Assignment/vaccin_covid.csv")
con = sqlite3.connect("Covid vaccines.db")
db = df.to_sql("Covid_vaccines", con, schema=None, if_exists="replace", index=True, index_label=None, chunksize=None, dtype=None, method=None)


In [7]:
#Skapa dataframes från queries i en linje kod
df = pd.read_sql_query("SELECT * FROM Covid_vaccines", con)
print(df.head())

   index      country iso_code        date  total_vaccinations  \
0      0  Afghanistan      AFG  2021-02-22                 0.0   
1      1  Afghanistan      AFG  2021-02-23                 NaN   
2      2  Afghanistan      AFG  2021-02-24                 NaN   
3      3  Afghanistan      AFG  2021-02-25                 NaN   
4      4  Afghanistan      AFG  2021-02-26                 NaN   

   people_vaccinated  people_fully_vaccinated  daily_vaccinations_raw  \
0                0.0                      NaN                     NaN   
1                NaN                      NaN                     NaN   
2                NaN                      NaN                     NaN   
3                NaN                      NaN                     NaN   
4                NaN                      NaN                     NaN   

   daily_vaccinations  total_vaccinations_per_hundred  \
0                 NaN                             0.0   
1              1367.0                             

In [14]:
cur.execute("SELECT vaccines FROM Covid_vaccines WHERE country = 'Sweden'")
cur.fetchall()

[('Moderna, Oxford/AstraZeneca, Pfizer/BioNTech',),
 ('Moderna, Oxford/AstraZeneca, Pfizer/BioNTech',),
 ('Moderna, Oxford/AstraZeneca, Pfizer/BioNTech',),
 ('Moderna, Oxford/AstraZeneca, Pfizer/BioNTech',),
 ('Moderna, Oxford/AstraZeneca, Pfizer/BioNTech',),
 ('Moderna, Oxford/AstraZeneca, Pfizer/BioNTech',),
 ('Moderna, Oxford/AstraZeneca, Pfizer/BioNTech',),
 ('Moderna, Oxford/AstraZeneca, Pfizer/BioNTech',),
 ('Moderna, Oxford/AstraZeneca, Pfizer/BioNTech',),
 ('Moderna, Oxford/AstraZeneca, Pfizer/BioNTech',),
 ('Moderna, Oxford/AstraZeneca, Pfizer/BioNTech',),
 ('Moderna, Oxford/AstraZeneca, Pfizer/BioNTech',),
 ('Moderna, Oxford/AstraZeneca, Pfizer/BioNTech',),
 ('Moderna, Oxford/AstraZeneca, Pfizer/BioNTech',),
 ('Moderna, Oxford/AstraZeneca, Pfizer/BioNTech',),
 ('Moderna, Oxford/AstraZeneca, Pfizer/BioNTech',),
 ('Moderna, Oxford/AstraZeneca, Pfizer/BioNTech',),
 ('Moderna, Oxford/AstraZeneca, Pfizer/BioNTech',),
 ('Moderna, Oxford/AstraZeneca, Pfizer/BioNTech',),
 ('Moderna, 

In [38]:
cur.execute("UPDATE Covid_vaccines SET total_vaccinations = 0 WHERE total_vaccinations = NULL")
cur.fetchall()

[]

In [37]:
con.commit()